In [1]:
import librosa
import os,glob,pickle
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf

In [3]:
from audiomentations import Compose, AddGaussianNoise, PitchShift, HighPassFilter

In [4]:
def featureExtractor(audio):
    librosa_audio_data, librosa_sample_rate=librosa.load(audio)
    mfccs_features= librosa.feature.mfcc(y=librosa_audio_data, sr= librosa_sample_rate,n_mfcc=50)
    ar=np.resize(feature,(50,280))
    ar = np.expand_dims(ar, axis=-1)
    return ar

In [5]:
def featureExtractor2(audio,sr):
    mfccs_features= librosa.feature.mfcc(y=audio, sr= sr,n_mfcc=50)
    return mfccs_features

In [67]:
x=[]
y=[]
def load_data():
    for file in glob.glob(r"D:\BTP\infant cry\donateacry_corpus_cleaned_and_updated_data\*"):
        label = os.path.basename(file)
        if label=='hungry':
            continue
        for audio in glob.glob(file+"\*.wav"):
            x.append(audio)
            y.append(label)


In [68]:
load_data()
print(len(x),len(y))

75 75


In [22]:
def shift_time_old(data, sampling_rate, shift_max, shift_direction):
    shift = np.random.randint(sampling_rate * shift_max)
    shift=min(shift,0.1*data.shape[0])
    if shift_direction == 'right':
        shift = -shift
    elif shift_direction == 'both':
        direction = np.random.randint(0, 2)
        if direction == 1:
            shift = -shift
    augmented_data = np.roll(data, shift)
    # Set to silence for heading/ tailing
    if shift > 0:
        augmented_data[:shift] = 0
    else:
        augmented_data[shift:] = 0
    return augmented_data

In [24]:
def shift_time(data, sampling_rate, shift_max, shift_direction):
    shift = np.random.randint(sampling_rate * shift_max)
    shift=min(shift,0.1*data.shape[0])
    if shift_direction == 'right':
        shift = -shift
    augmented_data = np.roll(data, shift)
    # Set to silence for heading/ tailing
    if shift > 0:
        augmented_data[:shift] = 0
    else:
        augmented_data[shift:] = 0
    return augmented_data

In [25]:
def time_stretch(signal, time_stretch_rate):
    return librosa.effects.time_stretch(signal, time_stretch_rate)


In [26]:
def pitch_scale(signal, sr, num_semitones):
    return librosa.effects.pitch_shift(signal, sr, num_semitones)

In [27]:
import random
def random_gain(signal, min_factor=0.8, max_factor=1.2):
    gain_rate = random.uniform(min_factor, max_factor)
    augmented_signal = signal * gain_rate
    return augmented_signal

In [28]:
# Raw audio augmentation
augment_raw_audio = Compose(
    [
        AddGaussianNoise(min_amplitude=0.01, max_amplitude=0.2, p=1),
        PitchShift(min_semitones=-8, max_semitones=8, p=1),
        HighPassFilter(min_cutoff_freq=2000, max_cutoff_freq=4000, p=1)
    ]
)


In [29]:
for audio,label in zip(x,y):
    signal,sr=librosa.load(audio)
    print(signal.shape[0])
    break
    

151263


In [33]:
aug_data=[]
aug_label=[]
i=0
for audio,label in zip(x,y):
        signal,sr=librosa.load(audio)
        augmented_signal1=shift_time(signal,sr,0.5,'left')
        aug_data.append((augmented_signal1,sr))
        aug_label.append(label)
        augmented_signal4=shift_time(signal,sr,0.5,'right')
        aug_data.append((augmented_signal4,sr))
        aug_label.append(label)
        
#         augmented_signal4=shift_time(signal,sr,3,'right')
#         if(label=='burping' or label=='belly_pain'):
        augmented_signal5=random_gain(signal,0.8,1.2)
        aug_data.append((augmented_signal5,sr))
        aug_label.append(label)

#         if(label=='burping'):
        augmented_signal2=time_stretch(signal,0.5)
        augmented_signal3=pitch_scale(signal,sr,2)
#         augmented_signal6=augment_raw_audio(signal,sr)
#         aug_data.append((augmented_signal6,sr))
#         aug_label.append(label)
        aug_data.append((augmented_signal2,sr))
        aug_label.append(label)
        aug_data.append((augmented_signal3,sr))
        aug_label.append(label)

In [32]:
# count=0
# for i in range(0,len(aug_label)):
#     if(aug_label[i]=='discomfort' and count<6):
#         del aug_data[i]
#         del aug_label[i]
#         count+=1
#     if count==6:
#         break

In [69]:
extracted_features=[]

def extractFeatures(X,Y):
    x=[]
    y=[]
    for audio,label in zip(X,Y):
        signal,sr=librosa.load(audio)
        feature = featureExtractor2(signal,sr)
        ar=np.resize(feature,(50,280))
        x.append(ar)
        y.append(label)
        extracted_features.append([feature,label])
    return x,y

In [70]:
X,Y=extractFeatures(x,y)

In [36]:
def extractFeatures2(X,Y):
    x=[]
    y=[]
    for audio,label in zip(X,Y):
        singal=audio[0]
        sr=audio[1]
        feature = featureExtractor2(signal,sr)
        ar=np.resize(feature,(50,280))
        x.append(ar)
        y.append(label)
        extracted_features.append([feature,label])
    return x,y

In [37]:
augX,augY=extractFeatures2(aug_data,aug_label)

In [38]:
for feature,label in zip(augX,augY):
    X.append(feature)
    Y.append(label)

In [71]:
print(len(X),len(Y))
c1=0
c2=0
c3=0
c4=0
for x in Y:
    if(x=='belly_pain'):
        c1+=1
    elif x=='burping':
        c2+=1
    elif x=='discomfort':
        c3+=1
    elif x=='tired':
        c4+=1;
print(c1,c2,c3,c4)

75 75
16 8 27 24


In [119]:
#train test split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=1)
print(len(X_train),len(y_train),len(X_test),len(y_test))

60 60 15 15


In [120]:
length=len(X_train)
X_train=np.array(X_train).reshape(length,50,280)
y_train=np.array(y_train)
print(X_train.shape)
print(y_train.shape)

(60, 50, 280)
(60,)


In [121]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y_train=to_categorical(labelencoder.fit_transform(y_train))
y_train.shape

(60, 4)

In [122]:
length=len(X_test)
X_test=np.array(X_test).reshape(length,50,280)
y_test=np.array(y_test)
print(X_test.shape)
print(y_test.shape)

(15, 50, 280)
(15,)


In [123]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y_test=to_categorical(labelencoder.fit_transform(y_test))
y_test.shape

(15, 4)

In [124]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
print(X_train.shape,X_test.shape)

(60, 50, 280, 1) (15, 50, 280, 1)


In [156]:
# BUILDING MODEL

import tensorflow as tf
# print(tf.__version__)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow import keras as keras
from sklearn import metrics

# num_labels=y.shape[1]

def build_model(input_shape):
    model=Sequential()
    # 1st conv layer
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu',input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((3,3), strides=(2,2),padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
    model.add(keras.layers.MaxPool2D((3,3), strides=(2,2),padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 3rd conv layer
    model.add(keras.layers.Conv2D(32, (2,2), activation='relu'))
    model.add(keras.layers.MaxPool2D((2,2), strides=(2,2),padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten into 1D array
    model.add(keras.layers.Flatten())
#     model.add(keras.layers.Dense(2048,activation='relu'))
#     model.add(keras.layers.Dense(1024,activation='relu'))
#     model.add(keras.layers.Dense(512,activation='relu'))
    model.add(keras.layers.Dense(32,activation='relu'))
    model.add(keras.layers.Dense(32,activation='relu'))
    model.add(keras.layers.Dropout(0.2))

    # output layer
    model.add(keras.layers.Dense(4))
    model.add(Activation('softmax'))
    return model

input_shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3])
model=build_model(input_shape)
model.summary()


Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_78 (Conv2D)           (None, 48, 278, 32)       320       
_________________________________________________________________
max_pooling2d_78 (MaxPooling (None, 24, 139, 32)       0         
_________________________________________________________________
batch_normalization_78 (Batc (None, 24, 139, 32)       128       
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 22, 137, 32)       9248      
_________________________________________________________________
max_pooling2d_79 (MaxPooling (None, 11, 69, 32)        0         
_________________________________________________________________
batch_normalization_79 (Batc (None, 11, 69, 32)        128       
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 10, 68, 32)      

In [157]:
# Training
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

optimizer=keras.optimizers.Adam(learning_rate=0.001)
# model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

start=datetime.now()
# history = model.fit(x_train, y_train, validation_split=0.3, epochs=10, verbose=0)
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=32,epochs=20)

duration=datetime.now() - start
print("Training completed in time: ",duration)


Epoch 1/20
2/2 [==============================] - 2s 611ms/step - loss: 0.7382 - acc: 0.2611 - val_loss: 1.2014 - val_acc: 0.3333
Epoch 2/20
2/2 [==============================] - 1s 366ms/step - loss: 0.5641 - acc: 0.3799 - val_loss: 1.2429 - val_acc: 0.1333
Epoch 3/20
2/2 [==============================] - 1s 372ms/step - loss: 0.4924 - acc: 0.4236 - val_loss: 1.0601 - val_acc: 0.1333
Epoch 4/20
2/2 [==============================] - 1s 333ms/step - loss: 0.4362 - acc: 0.5757 - val_loss: 0.9400 - val_acc: 0.2667
Epoch 5/20
2/2 [==============================] - 1s 366ms/step - loss: 0.4144 - acc: 0.6201 - val_loss: 0.8480 - val_acc: 0.2667
Epoch 6/20
2/2 [==============================] - 1s 361ms/step - loss: 0.4025 - acc: 0.6403 - val_loss: 0.7508 - val_acc: 0.4000
Epoch 7/20
2/2 [==============================] - 1s 359ms/step - loss: 0.3353 - acc: 0.7382 - val_loss: 0.6910 - val_acc: 0.3333
Epoch 8/20
2/2 [==============================] - 1s 366ms/step - loss: 0.3390 - acc: 0.77

In [158]:
loss, accuracy= model.evaluate(X_test, y_test, verbose=0)
# test_accuracy=model.evaluate(x_test,y_test,verbose=1)
# print(test_accuracy[1])
# test_accuracy
print("loss: ",loss)
# print("f1 score: ",f1_score)
# print("precision: ",precision)
print("accuracy: ",accuracy)
# print("recall: ",recall)

loss:  0.5291150808334351
accuracy:  0.6000000238418579


In [33]:
y_pred = model.predict(X_test) 
y_pred = np.argmax(y_pred, axis = 1) 
label = np.argmax(y_test,axis = 1) 

from sklearn.metrics import confusion_matrix
print(confusion_matrix(label,y_pred))
# print("y_test:\n", label)
# print("y_pred:\n",y_pred) 


[[ 0  0 22  0]
 [ 0  0  9  0]
 [ 0  0 25  1]
 [ 0  0 32  0]]


In [34]:
predicted_label=model.predict_classes(X_test)
print(predicted_label)
predcited_class=labelencoder.inverse_transform(predicted_label)
predcited_class

C:\Users\Rajeen\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3]


array(['discomfort', 'discomfort', 'discomfort', 'discomfort',
       'discomfort', 'discomfort', 'discomfort', 'discomfort',
       'discomfort', 'discomfort', 'discomfort', 'discomfort',
       'discomfort', 'discomfort', 'discomfort', 'discomfort',
       'discomfort', 'discomfort', 'discomfort', 'discomfort',
       'discomfort', 'discomfort', 'discomfort', 'discomfort',
       'discomfort', 'discomfort', 'discomfort', 'discomfort',
       'discomfort', 'discomfort', 'discomfort', 'discomfort',
       'discomfort', 'discomfort', 'discomfort', 'discomfort',
       'discomfort', 'discomfort', 'discomfort', 'discomfort',
       'discomfort', 'discomfort', 'discomfort', 'discomfort',
       'discomfort', 'discomfort', 'discomfort', 'discomfort',
       'discomfort', 'discomfort', 'discomfort', 'discomfort',
       'discomfort', 'discomfort', 'discomfort', 'discomfort',
       'discomfort', 'discomfort', 'discomfort', 'discomfort',
       'discomfort', 'discomfort', 'discomfort', 'disco